# Final Project: Overview

# Objective

The objective of this project is for you to demonstrate your mastery of the Machine Learning process
**using Neural Networks**.



# Submission requirements

The guidelines will be similar to the Midterm
- you will write a procedure that takes raw data and produces predictions

You will submit a *single* model for evaluation.

**Demonstrate that all cells in your notebook work**

The final cell in your notebook should print the message "Done"
- `print("Done")`
- If we run your notebook and this last cell does not execute your submission will be inadequate

## Testing

*You must perform out of sample testing*.

If you want to perform cross-validation in training, that is fine, but you
must *also* test out of sample to show that you are not over-fitting.

It is up to you to create the out of sample data that you feel best evaluates your model.

We will create holdout data (that we will not show you) for grading.

The procedure you write to make predictions should be able to work on the unseen holdout data
(similar to how it should work for your test set but the holdout set has *no targets*)

    

# The data

Data will be provided to you 
- as multiple files in a directory which we refer to as a *data directory*

The reason for this is that the different files may convey different information.

You will be responsible for deciding
- which files to use
- which fields within the files to use

We will give you a data directory for training.

# Submission guidelines

Here are the basics, a code template that you must complete is in the following cells
- you will be required to store  your model in a file
- you will be required to write a procedure `MyModel` that takes two arguments
    - `test_dir`
        - this is a *relative path* to the holdout data directory
    - `model_path`
        - this is a *relative path* to the file containing your model
- the holdout data directory is similar in structure to the training data directory
    - but without target labels !  It is your job to predict these.
- your procedure must produce predictions given this holdout data directory

This means that your procedure must
- prepare the files in the holdout data directory similar to the way that they were prepared in the training data directory

We will provide you with a sample data directory that will resemble the holdout -- this is so that you
may test the procedure you write for submission.



## Detailed submission guidelines


In **addition to your notebook that trains/evaluates your model**, 
- please also submit an **archive file of the directory** whose name is stored in `model_path`, which 
contains your trained model.
    - use `saveModel` to put your final, trained model in this directory
- We will **not** train your model; we will only use the method `MyModel`
    - which **you** will implement
    - and which uses `loadModel` and the directory whose name is stored in `model_path`
    - this will create the model that we will evaluate


Here is a code template for you to complete
- it will save your model (assuming it is in variable `my_model`)
- it provides the specification for procedure `MyModel`, which *you must complete*


packages

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM,Dropout,BatchNormalization,Reshape
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [1]:
def addPositive(x):
    return x[x>0].sum()

def addNegative(x):
    return -x[x<0].sum()

In [2]:
def get_technical_indicators(dataset):
   # Create 7 and 21 days Moving Average
   dataset['ma7'] = dataset['Close'].rolling(window=7).mean()
   dataset['ma21'] = dataset['Close'].rolling(window=21).mean()
   
   # Create MACD
   dataset['26ema'] = dataset['Close'].ewm(span=26).mean()
   dataset['12ema'] = dataset['Close'].ewm(span=12).mean()
   dataset['MACD'] = (dataset['12ema']-dataset['26ema'])

   # Create Bollinger Bands
   dataset['20sd'] = dataset['Close'].rolling(window=20).std()
   dataset['upper_band'] = dataset['ma21'] + (dataset['20sd']*1.96)
   dataset['lower_band'] = dataset['ma21'] - (dataset['20sd']*1.96)
   
   # Create Exponential moving average
   dataset['ema'] = dataset['Close'].ewm(com=0.5).mean()
   
   # Create Momentum
   dataset['momentum'] = dataset['Close']-dataset['Close'].shift(10)
    
   # Create Average True Range
   dataset['HLrange'] = (dataset['High']-dataset['Low']).abs()
   dataset['HCrange'] = (dataset['High']-dataset['Close'].shift(1)).abs()
   dataset['CLrange'] = (dataset['Close'].shift(1)-dataset['Low']).abs()
   dataset['TR'] = dataset.loc[:,['HLrange','HCrange','CLrange']].max(axis=1)
   dataset['ATR'] = dataset['TR'].rolling(window=20).mean()
    
   # Create Volume Ratio
   dataset['Vlchange'] = dataset['Volume'] - dataset['Volume'].shift(1)
   dataset['VR'] = dataset['Vlchange'].rolling(window=14).apply(addPositive)/dataset['Vlchange'].rolling(window=14).apply(addNegative)
   
   # Create William Ratio
   dataset['WR'] =  (dataset['High'].rolling(window=14).max()-dataset['Close']) \
    /(dataset['High'].rolling(window=14).max()-dataset['Low'].rolling(window=14).min())*100
    
   # Create Money Flow Index
   dataset['MoneyFlow'] = dataset.loc[:,['High','Low','Close']].mean(axis=1)*dataset['Volume']
   dataset['Mfchange'] = dataset['MoneyFlow'] - dataset['MoneyFlow'].shift(1)
   dataset['MFI'] = dataset['Vlchange'].rolling(window=14).apply(addPositive)/dataset['Vlchange'].rolling(window=14).apply(addNegative)
   
   # Create Open-Close Range
   dataset['OCR'] = dataset['Open']-dataset['Close'].shift(1)

   return dataset

In [4]:
def create_rolled_data(X,y,rolling_num=30):
    X_roll = []
    y_roll = []
    for i in range(rolling_num, X.shape[0]):
        X_roll.append(X[i-rolling_num:i,:])
        y_roll.append(y[i, 0])
    X_roll, y_roll = np.array(X_roll), np.array(y_roll)

    print(X_roll.shape)
    print(y_roll.shape)

    # Reshaping
    X_roll = np.reshape(X_roll, (X_roll.shape[0], X_roll.shape[1], X_roll.shape[2]))
    y_roll = np.reshape(y_roll,(y_roll.shape[0],1))
    
    return X_roll,y_roll

In [15]:
import os
from tensorflow.keras.models import load_model

modelName = "final_model"
model_path = os.path.join(".", modelName)

def saveModel(model, model_path): 
    try:
        os.makedirs(model_path)
    except OSError:
        print("Directory {dir:s} already exists, files will be over-written.".format(dir=model_path))
        
    # Save JSON config to disk
    json_config = model.to_json()
    with open(os.path.join(model_path, 'config.json'), 'w') as json_file:
        json_file.write(json_config)
    # Save weights to disk
    model.save_weights(os.path.join(model_path, 'weights.h5'))
    
    print("Model saved in directory {dir:s}; create an archive of this directory and submit with your assignment.".format(dir=model_path))
    
def loadModel(model_path):
    # Reload the model from the 2 files we saved
    with open(os.path.join(model_path, 'config.json')) as json_file:
        json_config = json_file.read()
    model = tf.keras.models.model_from_json(json_config)
    model.load_weights(os.path.join(model_path, 'weights.h5'))
    
    return model

def MyModel(test_dir, model_path):
    # YOU MAY NOT change model after this statement !
    model = loadModel(model_path)
    
    # It should run model to create an array of predictions; we initialize it to the empty array for convenience
    predictions = []
    
    # We need to match your array of predictions with the examples you are predicting
    # The array below (ids) should have a one-to-one correspondence and identify the example your are predicting
    # For Bankruptcy: the Id column
    # For Stock prediction: the date on which you are making a prediction
    ids = []
    
    # read data
    data = pd.read_csv(test_dir+'\AAPL.csv',parse_dates=True, index_col = "Dt")
    data = data.iloc[:,[0,1,4,5,6,7]]
    
    #prepare the data
    data_new = get_technical_indicators(data)
    data_new['Return'] = data_new['Close'].pct_change()
    data_new = data_new.dropna()
    
    X_test = data_new.loc[:,['High', 'Low', 'Open', 'Volume','ma7', 'ma21','26ema', '12ema', 'MACD','upper_band', 'lower_band', 'ema',
       'momentum','ATR','VR', 'WR','MFI', 'OCR']]
    y_test = data_new.loc[:,['Return']]
    
    sc_x = MinMaxScaler()
    sc_y = MinMaxScaler()
    X_test = sc_x.fit_transform(X_test)

    y_test = sc_y.fit_transform(y_test)
    
    rolling_num=15
    X_test_roll,y_test_roll = create_rolled_data(X_test,y_test,rolling_num=rolling_num)
    ids = list(data_new.index[rolling_num:])
    predicted = model.predict(X_test_roll)
    predicted_stock_return = sc_y.inverse_transform(predicted)
    
    predictions = list(predicted_stock_return.T[0])
    #print(len(predictions),len(ids))
    
    return predictions, ids

# Assign to variable my_model the model that is your final model (the one  you will be evaluated on)
#my_model = None # CHANGE None to your model !

#saveModel(my_model, model_path)

## Evaluate your model on the holdout data directory

**You must run the following cell** from the directory that contains your model file

Here is how we will evaluate your submission
- we will create a directory whose only content is
    - sub-directory `Data`
- we will copy your model file to this directory with the name stored in `model_path`
- we will run the cell in your notebook that should be a copy of the one below
    - it calls procedure `MyModel` with the arguments given below
    - your implementation of `MyModel`
        - must successfully load your model file, *given where **we** have place it as described above*
        - must successfully return one prediction for each example in the holdout directory *given where **we** have placed the holdout directory*

In [18]:
#holdout_dir = os.path.join(".", "Data", "sample")
#predicts = MyModel(holdout_dir, model_path)

(216, 15, 18)
(216,)


In [ ]:
holdout_dir = os.path.join(".", "Data", "holdout")
predicts = MyModel(holdout_dir, model_path)

In [17]:
print("Done")

Done
